In [ ]:
import pandas as pd
from pathlib import Path

# Set data paths

In [ ]:
# Explore data sources from .xslsx files, merge them and save them as .csv

ach_path = Path("data/2_Achmad/achmed_raw.csv")
wal_path = Path("data/3_Waluyo/waluyo_raw.csv")
old_brg_path = Path("data/4_brg_old/brg_old.csv")

DATA_COLS = ["source", "id", "lon", "lat", "date", "gwl_cm"]

In [ ]:
ach_df = pd.read_csv(ach_path)

# 1. Read data

### BRGM new data

In [ ]:
ach_cols_rename = {
    "gwl_rata" : "gwl_cm",
}

# rename columns
ach_df.rename(columns=ach_cols_rename, inplace=True)

# Convert date column to datetime
ach_df["date"] = pd.to_datetime(ach_df["date"])

# multiply gwl_cm by 100 to convert it to cm
ach_df["gwl_cm"] = ach_df["gwl_cm"] * 100

# set a new column for source
ach_df["source"] = "ach"

ach_df = ach_df[DATA_COLS]
ach_df.head()

In [ ]:
ach_df.id.unique()

### PKEG data

In [ ]:
wal_cols_rename = {
    "date(dd/mm/yyyy)" : "date",
    "gwl(cm)" : "gwl_cm",
    "coor_y(dd)" : "lat",
    "coor_x(dd)" : "lon",
}

# Read Waluyo's data
wal_df = pd.read_csv(wal_path, sep=";")

# Rename columns
wal_df.rename(columns=wal_cols_rename, inplace=True)

# combine "kode_perusahaan"	and "kode_titik" to create a unique id
wal_df.loc[:, "id"] = wal_df["kode_perusahaan"] + "_" + wal_df["kode_titik"]

# Convert date column to datetime
wal_df.loc[:, "date"] = pd.to_datetime(wal_df["date"], dayfirst=True)

# set a new column for source
wal_df["source"] = "wal"

# Only select columns that are needed
wal_df = wal_df[DATA_COLS]
wal_df.head()

### Previous BRG data

In [ ]:
old_brg_df = pd.read_csv(old_brg_path)
old_brg_df.loc[:, "source"] = "old_brg"
old_brg_df.loc[:, "date"] = pd.to_datetime(old_brg_df["date"], dayfirst=True)

# Multiply gwl_cm by 100 to convert it to cm
old_brg_df["gwl_cm"] = old_brg_df["gwl_cm"] * 10
old_brg_df = old_brg_df[DATA_COLS]
old_brg_df.head()

### Concatenate all data sources

In [ ]:
# Merge the two dataframes
df = pd.concat([ach_df, wal_df, old_brg_df], ignore_index=True)

# save the dataframe as csv
df.to_csv("data/field_data_all_with_old.csv", index=False)

In [ ]:
# Remove duplicate dates for each id using the mean value
# Group by 'id' and 'date' and calculate the mean value for 'gwl_cm' while keeping other columns

print("Before removing duplicates", len(df))

agg_dict = {'source':'first','lon':'first','lat':'first','gwl_cm':'mean'}
df = df.groupby(['id','date']).agg(agg_dict).reset_index()

print("After removing duplicates", len(df))

In [ ]:
# Below I will remove the duplicated coordinates IDS and keep the first one
# get unique lon-lat pairs
unique = df[["id", "lon", "lat"]].drop_duplicates()

# Get duplicated lon-lat pairs
duplicated = unique[unique.duplicated(subset=["lon", "lat"], keep=False)]

duplicated = duplicated.drop_duplicates(subset=["lon", "lat"], keep="first")

# Get the duplicated ids
duplicated_ids = duplicated["id"].unique()

# # get dataframe without duplicated ids

df = df[~df["id"].isin(duplicated_ids)]

df.to_csv("data/field_data_unique_coords.csv", index=False)
df

In [ ]:
# get unique coordinates for each station
stations = df[["id", "source", "lon", "lat"]].drop_duplicates()


# Convert to GeoDataFrame
from geopandas import GeoDataFrame
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(stations.lon, stations.lat)]
stations_gdf = GeoDataFrame(stations, geometry=geometry)
stations_gdf.crs = "EPSG:4326"
stations_gdf.to_file("data/0_shp/unique_stations_no_repeated.shp")